Result could be different . because of some under fitting problem .if you run two time then you can understand the problem.public score beetween (.83 to 93)

In [ ]:
import pandas as pd

Train=pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/train.csv')
Test= pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/test.csv')
sample_submission= pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/sample_submission.csv')

In [ ]:
#replace ".."with .100
Train=Train.replace('..', 0.102)  
Test=Test.replace('..', 0.102)  

In [ ]:
#Unique country for iteration
unique = Train['Country Name'].unique()

In [ ]:
#trainig and inferencing with LogisticRegression and ARIMA
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LogisticRegression
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
j=[]
for i in unique:
    transposed_df=0
    fg = Train[Train['Country Name'] == i].reset_index(drop=True)
    columns_to_drop = ['Country Code', 'Country Name','Indicator']
    fg = fg.drop(columns_to_drop, axis=1)
    transposed_df = fg.transpose()
    #print(fg)
    #print('done for now')
    Xtrain = transposed_df[[0,1, 2,3,4,5,6,7,8,9,10]]
    Xtrain
    Ytrain=transposed_df[11]
    Ytrain
    logistic_regression = LogisticRegression()
    try:
        logistic_regression.fit(Xtrain,Ytrain)
        fgt = Test[Test['Country Name'] == i].reset_index(drop=True)
        columns_to_drop = [ 'Country Name','Indicator']
        fgt = fgt.drop(columns_to_drop, axis=1)
        transposed_df = fgt.transpose()
        #print(transposed_df)
  
        Xtest = transposed_df[[0,1, 2,3,4,5,6,7,8,9,10]]
        forecast = logistic_regression.predict(Xtest.astype(float))
        
        model = ARIMA(Ytrain.astype(float), order=(0,1,3))  # Adjust order as needed
        model_fit = model.fit()

    
        forecast2 = model_fit.forecast(steps=16)
        
        row_index = sample_submission[sample_submission.eq(i).any(axis=1)].index[0]


        new_values = [i, forecast[0], forecast[1],forecast[2], forecast[3], forecast[4],forecast2[30]]
        sample_submission.loc[row_index] = new_values
    except:
        j.append(i)
    

In [ ]:
#trainig and inferencing with LogisticRegression and ARIMA
import warnings
import tensorflow as tf
warnings.filterwarnings("ignore")
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout


k=0
for i in unique:
    transposed_df=0
    fg = Train[Train['Country Name'] == i].reset_index(drop=True)
    columns_to_drop = ['Country Code', 'Country Name','Indicator']
    fg = fg.drop(columns_to_drop, axis=1)
    transposed_df = fg.transpose()
    #print(fg)
    #print('done for now')
    Xtrain = transposed_df[[0,1, 2,3,4,9,10]].astype(float)
    xt=Xtrain.values
    Ytrain=transposed_df[11].astype(float) 
    yt=Ytrain.values
    
    if k == 0:
        k+=1
        model = Sequential([
              LSTM(64, return_sequences=True, input_shape=(Xtrain.shape[1], 1)),
              Dropout(0.2),  # Adding dropout for regularization
              LSTM(64, return_sequences=True),
              Dropout(0.2),
              LSTM(64),  # Last LSTM layer without return sequences
              Dropout(0.2),
              Dense(32, activation='relu'),  # Adding a dense layer for additional processing
              Dense(1)  # Output layer
              ])
        model.compile(optimizer = tf.keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError())
        Xtrain = xt.reshape((xt.shape[0], xt.shape[1], 1))
        model.fit( Xtrain,  yt, epochs=22, batch_size=4)
        model.save("/kaggle/working/my_lstm_model.h5")
    else:
        loaded_model = tf.keras.models.load_model("/kaggle/working/my_lstm_model.h5")
        loaded_model.compile(optimizer = tf.keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError())
        loaded_model.fit(Xtrain, yt, epochs=22, batch_size=4)
        loaded_model.save("/kaggle/working/my_lstm_model.h5")

In [ ]:
#predicting and forecasting unknown Co2 Value
for i in j:
    fg = Test[Test['Country Name'] == i].reset_index(drop=True)
    columns_to_drop = ['Country Name','Indicator']
    fg = fg.drop(columns_to_drop, axis=1)
    transposed_df = fg.transpose()
    #print(fg)
    #print('done for now')
    XTest = transposed_df[[0,1, 2,3,4,9,10]]
    loaded_model = tf.keras.models.load_model("/kaggle/working/my_lstm_model.h5")
   
    forecast = loaded_model.predict(XTest.astype(float))
    df = pd.DataFrame(forecast)
    model2 = ARIMA(df[0], order=(0,1,3))  # Adjust order as needed
    model_fit = model2.fit()

    
    forecast2 = model_fit.forecast(steps=16)
        
    row_index = sample_submission[sample_submission.eq(i).any(axis=1)].index[0]


    new_values = [i, forecast[0][0], forecast[1][0],forecast[2][0], forecast[3][0], forecast[4][0],forecast2[14]]
    sample_submission.loc[row_index] = new_values


In [ ]:
 sample_submission.head(15)

In [ ]:
sample_submission.to_csv('submission.csv', index=False)